# How to upgrade existing AmazonForecast predictor to AutoPredictors

Helps advanced users start with Amazon Forecast Upgrade to AutoPredictor Feature. 

Predictors created with AutoML or manual selection can be upgraded to an AutoPredictor. Upgrading an existing to AutoPredictor will transfer all the relevant predictor configuration settings and create an AutoPredictor using the same datasets.

After Upgrading to AutoPredictor, the original predictor will remain active and the upgraded predictor will have a separate Predictor ARN. This enables you to compare accuracy metrics between the two predictors, and you can still generate forecasts with the original predictor.

This allows users to conviently upgrade existing predictor(s) without having the need to pass all the exiting configurations.

The demo notebook provides guidance to upgrade an existing Predictors which were created via CreatePredictor API to upgrade to more advances AutoPredictor which offer various benifits. Refer  https://docs.aws.amazon.com/forecast/latest/dg/howitworks-predictor.html for details.

Prerequisites: 
[AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/installing.html) . 

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

**Note that this notebook is meant for customers who already have existing Predictors and want to move to AutoPredictor for the added functionalities.**


## Table Of Contents
* [Setting up](#setup)
* [Check Predictors eligible for Upgrade](#upgradableCheck)
* [Upgrading to AutoPredictor](#upgradeToAuto)

**Read Every Cell FULLY before executing it**


## Setup <a class="anchor" id="setup"></a>

In [ ]:
import sys
import os
import time

import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

In [ ]:
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region)
BETA = 'https://seer-beta-service.us-west-2.amazonaws.com'
forecast = session.client(service_name='forecast-auto', endpoint_url=BETA)
#forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

## Check Predictors eligible for Upgrade<a class="anchor" id="upgradableCheck"></a>


In this section, we go over the Predictors in your account to see if a Predictor is upgradable and if there is, we upgrade that Predictor.

Here we list the Predictor for your account. 

In [ ]:
li=forecast.list_predictors()['Predictors']
li

Now we look for a Predictor for which is eligible for upgrade.

Notice that **IsAutoPredictor** is **False** for Predictors created earlier through CreatePredictor API

All such Predictors are eligible for upgrade using the new [CreateAutoPredictor API]()

In [ ]:
upgradablePredictor = next((x for x in li if x["IsAutoPredictor"] == False), None)

If no predictor is elegible for upgrade, we terminate the notebook exeuction

In [ ]:
if upgradablePredictor == None:
    print("No predictor is eligible for upgrade! You can create a new AutoPredictor using https://github.com/aws-samples/amazon-forecast-samples/tree/main/notebooks/basic/Getting_Started")
    sys.exit("Not executing anything further!")

Below we list the Predictor Arn which is eligible for Upgrade and describe that to see the Predictor charecteristics.

In [ ]:
upgradablePredictor

In [ ]:
forecast.describe_predictor(PredictorArn=upgradablePredictor["PredictorArn"])

## Upgrading to AutoPredictor <a class="anchor" id="upgradeToAuto"></a>

In this section, we pick the existing predictor selected in the previous section and upgrate that to an AutoPredictor.

In [ ]:
project = 'ws_upgrade_predictor' # Replace this with a unique name here, make sure the entire name is < 30 characters.

In [ ]:
newUpgradePredictorName=project

For Upgrading the predictor, we pass the referencePredictor Arn an an input to CreateAutoPredictor API. You can pass additional inputs like tags but rest of the predictor configuration cannot be changed.

Here referencePredictorArn will be the predictorArn which is upgradable above.

In [ ]:
referencePredictorArn=upgradablePredictor["PredictorArn"]

print("ReferencePredictorArn: ", referencePredictorArn)

Upgrading an existing Predictor is very simple and does not require a lot of input.

Amazon Forecast takes the old predictor as a reference and creates a new AutoPredictor using the datasets for the existing predictor.

In [ ]:
create_auto_predictor_response=forecast.create_auto_predictor(PredictorName=newUpgradePredictorName, 
                                                              ReferencePredictorArn=referencePredictorArn)

In [ ]:
upgradedPredictor=create_auto_predictor_response['PredictorArn']

In [ ]:
print("New Predictor Arn: ", upgradedPredictor)

Now we wait for the new AutoPredictor to complete and it can take upto 2-3 hours.

In [ ]:
print("Waiting for Predictor creation to complete.")

status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=upgradedPredictor))

describe_auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=upgradedPredictor)
print(f"\n\nThe Predictor with ARN **{upgradedPredictor}** is now **{describe_auto_predictor_response['Status']}**.")


Here we describe the new AutoPredictor Accuracy metrics and details about the auto predictor

In [ ]:
describe_auto_predictor_response

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=upgradedPredictor)

# Cleanup

Once we have completed the above steps, we can start to cleanup the resources we created. All the created resources can be deleted using `delete_resource_tree` and it is an asynchronous operation, so we have added the helpful `wait_till_delete` function. To learn more about deleting a parent resource and all its child resources, visit [DeleteResourceTree](https://docs.aws.amazon.com/forecast/latest/dg/API_DeleteResourceTree.html) API. 
Resource Limits documented <a href="https://docs.aws.amazon.com/forecast/latest/dg/limits.html">here</a>.

Note that below cells have been intentionally commented out to prevent resource from being deleted. Uncomment below cells for deleting resources.

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_resource_tree(ResourceArn=upgradedPredictor))